In [1]:
import pandas as pd

df = pd.read_csv("../datasets/feature_train.csv")
df.head()

,ts_code,trade_date,open,high,low,close,vol,amount,return_rate,macd,...,pe_ttm,lncap,turnover_rate_rolling,roe,debt_to_assets,q_gr_yoy,sh_chg,alpha,beta,var_e
0,600000.SH,2000-02-29,3.22,3.27,3.17,3.22,109559.0,275768.296,0.0031,0.001481,...,70.5807,15.618205,3.623260,6.5944,92.6751,53.0924,0.005703,0.006491,0.605899,0.000568
1,600000.SH,2000-03-01,3.22,3.47,3.22,3.30,417051.0,1096015.251,0.0248,0.013680,...,72.4034,15.643702,4.202110,6.5944,92.6751,53.0924,-0.009818,0.006919,0.594308,0.000609
2,600000.SH,2000-03-02,3.30,3.32,3.21,3.25,129999.0,330633.290,-0.0152,0.014678,...,71.2817,15.628088,4.325320,6.5944,92.6751,53.0924,0.008994,0.006530,0.587057,0.000603
3,600000.SH,2000-03-03,3.27,3.41,3.27,3.32,161098.0,420418.261,0.0215,0.022851,...,72.9081,15.650649,4.449870,6.5944,92.6751,53.0924,0.014598,0.005095,0.616446,0.000565
4,600000.SH,2000-03-06,3.39,3.44,3.27,3.27,206162.0,542130.542,-0.0151,0.020702,...,71.7864,15.635145,4.627465,6.5944,92.6751,53.0924,-0.032759,0.004717,0.617602,0.000551


In [2]:
import numpy as np
import matplotlib.pyplot as plt


def get_diff(d, ratio=0.01):
    result, w_ = [], 1
    sum_, len_, ratio_ = 0, 0, ratio+1
    while ratio_ > ratio:
        result.append(w_)
        len_ += 1
        w_ = -result[-1]/len_*(d-len_+1)
        sum_ += abs(w_)
        ratio_ = abs(w_)/sum_
    return result


def get_diff_list(ratio=0.01):
    results = {}
    for d in np.arange(0.01, 4, 0.01):
        results[d] = get_diff(d, ratio=ratio)
    return results


d_results = get_diff_list()


tmp = pd.DataFrame()
tmp["d"] = [i[0] for i in d_results.items()]
tmp["length"] = [len(i[1]) for i in d_results.items()]

plt.plot(tmp["d"], tmp["length"])
plt.title("The length of w")
plt.xlabel("d")
plt.ylabel("length")
plt.show()

<Figure size 640x480 with 1 Axes>

In [3]:
cols = df.columns[2:]
cols

Index(['open', 'high', 'low', 'close', 'vol', 'amount', 'return_rate', 'macd',
       'roc', 'middleboll', 'upboll', 'downboll', 'cci', 'rsi', 'wr', 'k', 'd',
       'j', 'pe_ttm', 'lncap', 'turnover_rate_rolling', 'roe',
       'debt_to_assets', 'q_gr_yoy', 'sh_chg', 'alpha', 'beta', 'var_e'],
      dtype='object')

In [4]:
from statsmodels.tsa.stattools import adfuller

# 二分查找


def binary_search(f, x0, x1, max_xe=1e-10, max_fe=1e-10):
    if abs(f(x0)) < max_fe:
        return x0
    if abs(f(x1)) < max_fe:
        return x1

    assert f(x0) * f(x1) < 0
    xe, mid = np.inf, np.inf
    while (abs(xe) > max_xe) and (abs(mid) > max_fe):
        left = f(x0)
        mid = f((x0 + x1) / 2)
        xe = (x1 - x0) / 2
        if mid * left < 0:
            x1 = (x0 + x1) / 2
        else:
            x0 = (x0 + x1) / 2
    else:
        return (x0 + x1) / 2


def loss_function_adf(sr):
    def loss_function_adf_(d):
        d_sr = get_diff(d)
        sr_ = sr.rolling(len(d_sr)).apply(lambda x: (x[::-1]*d_sr).sum())
        sr_.dropna(inplace=True)
        p = adfuller(sr_, maxlag=1, regression='c', autolag=None)[1]
        return p-0.05
    return loss_function_adf_


d_dict = {}
for col in cols:
    sr = df[col]
    # 注意autolag=None
    df2 = adfuller(sr, maxlag=1, regression='c', autolag=None)
    p = df2[1]
    if p < 0.05:
        # 代表不需要差分
        d_dict[col] = 0
        continue
    else:
        d = binary_search(loss_function_adf(sr), 0.1, 5)
        d_dict[col] = d
print(d_dict)

f:\mypython\python3.7\lib\site-packages\ipykernel_launcher.py:29: FutureWarning: Currently, 'apply' passes the values as ndarrays to the applied function. In the future, this will change to passing it as Series objects. You need to specify 'raw=True' to keep the current behaviour, and you can pass 'raw=False' to silence this warning


{'open': 0.23188335779414043, 'high': 0.25329646693717217, 'low': 0.22684944970969814, 'close': 0.22875516292260725, 'vol': 0, 'amount': 0, 'return_rate': 0, 'macd': 0, 'roc': 0, 'middleboll': 3.227762526417064, 'upboll': 0.6002883901790481, 'downboll': 0.4749730807583546, 'cci': 0, 'rsi': 0, 'wr': 0, 'k': 0, 'd': 0, 'j': 0, 'pe_ttm': 0.18350141577611795, 'lncap': 0.3659727364152787, 'turnover_rate_rolling': 0, 'roe': 0, 'debt_to_assets': 0.20389189351408277, 'q_gr_yoy': 0, 'sh_chg': 0, 'alpha': 0, 'beta': 0, 'var_e': 0}


In [5]:
tmp = pd.DataFrame()
tmp["Index"] = [i for i, j in d_dict.items()]
tmp["d"] = [j for i, j in d_dict.items()]
tmp[tmp["d"] > 0].reset_index(drop=True)

,Index,d
0,open,0.231883
1,high,0.253296
2,low,0.226849
3,close,0.228755
4,middleboll,3.227763
5,upboll,0.600288
6,downboll,0.474973
7,pe_ttm,0.183501
8,lncap,0.365973
9,debt_to_assets,0.203892


In [6]:
tmp["d_is_zero"] = True
tmp.loc[tmp["d"] > 0, "d_is_zero"] = False
pd.DataFrame(tmp["d_is_zero"].value_counts())

,d_is_zero
True,18
False,10


In [7]:
# 进行差分
for col, d in d_dict.items():
    if abs(d) < 0.01:
        continue
    d_sr = get_diff(d)
    sr = df[col]
    sr_ = sr.rolling(len(d_sr)).apply(
        lambda x: (x[::-1]*d_sr).sum(), raw=False)
    df[col] = sr_

df.dropna(inplace=True)
df.reset_index(drop=True, inplace=True)
df.head()

,ts_code,trade_date,open,high,low,close,vol,amount,return_rate,macd,...,pe_ttm,lncap,turnover_rate_rolling,roe,debt_to_assets,q_gr_yoy,sh_chg,alpha,beta,var_e
0,600000.SH,2000-03-27,1.423685,1.347981,1.439653,1.473311,61611.0,158738.686,0.0153,0.007468,...,37.466462,4.297805,3.696540,6.5944,43.792379,53.0924,0.026244,0.002000,0.638360,0.000220
1,600000.SH,2000-03-28,1.463812,1.356395,1.416104,1.410052,70742.0,180806.578,-0.0151,0.006794,...,30.914936,4.275985,3.635890,6.5944,43.792379,53.0924,0.008743,0.002128,0.626859,0.000228
2,600000.SH,2000-03-29,1.392627,1.286466,1.388263,1.376263,74952.0,189202.622,-0.0123,0.000914,...,31.140015,4.267056,3.101365,6.5944,43.792379,53.0924,-0.001470,0.003747,0.786882,0.000240
3,600000.SH,2000-03-30,1.362020,1.303048,1.425140,1.416451,44008.0,111744.655,0.0093,0.000868,...,32.218542,4.280112,2.967005,6.5944,43.792379,53.0924,0.012398,0.002711,0.851317,0.000212
4,600000.SH,2000-03-31,1.392931,1.288517,1.400378,1.393524,43204.0,109090.852,-0.0062,-0.001806,...,32.014961,4.272451,2.782795,6.5944,43.792379,53.0924,-0.005941,0.002952,0.887382,0.000225


In [8]:
import pickle
df.to_csv("../datasets/ADF_adjust_train.csv", index=False)

with open("../datasets/d_dict.pkl", "wb+") as f:
    pickle.dump(d_dict, f)